# setting kernel type

to use tpu  set training = 'tpu'   for gpu  training = 'gpu'   for cpu training = 'cpu'

# **Note for TPU
**while these notebook is ok to work in tpu  ,  the xla for pytorch As per they have announced, its still under development and so for operations not supported they automatically switch to CPU so it might get even slower than GPU.

so for some reson in these notebook when using this data sets where  we have  large size of images when  image resize and transformations  it will use cpu for that transformation and do training in tpu and these will cost memory to work around with these either you save images after transformation and read them diractly into the model or  i just skip val phase and also  make batches very small in tpu 
to  use tpu in these dataset
just set profile='low'  other  can set profile=''high'

In [ ]:
training = 'gpu'
profile =  'high'  # for this dtata set to use tpu set these var to low

# install lighting and xla for tpu

In [ ]:
if training =='tpu':
    !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
    !python pytorch-xla-env-setup.py --version 1.7 --apt-packages libomp5 libopenblas-dev

In [ ]:
! pip install pytorch-lightning  #==1.1.5


In [ ]:
!pip install timm

# library  needed by lightining

In [ ]:


import torch
from torch.nn import functional as F
from torch import nn
from pytorch_lightning.core import LightningModule
from sklearn.metrics import label_ranking_average_precision_score

print(torch.__version__)

# other libraarys

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from torchvision import datasets, models, transforms
from torch import nn, optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms  
from torch.utils.data.sampler import SubsetRandomSampler  
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import torch.nn.functional as F

import time
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics

import timm
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint , EarlyStopping


# seeding everthing

In [ ]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if training == 'gpu':
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
seed_everything(1001)

# configartions

In [ ]:
DATA_PATH = "../input/ranzcr-clip-catheter-line-classification"
TRAIN_PATH = "../input/ranzcr-clip-catheter-line-classification/train/"
TEST_PATH = "../input/ranzcr-clip-catheter-line-classification/test/"
MODEL_PATH = (
    "../input/vit-base-models-pretrained-pytorch/jx_vit_base_p16_224-80ecf9dd.pth"
)



IMG_SIZE = 384
if profile =='low':
    BATCH_SIZE = 8 
    val_BATCH_SIZE = 2
    valchecking = 3
    epoch = 2
    monit = 'train_loss'
else:
    BATCH_SIZE = 16
    val_BATCH_SIZE = 16
    valchecking = 1
    epoch = 20
    monit = 'val_loss'

# read csv file with pandas

In [ ]:
df = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
df.head()

# split data

In [ ]:
train_df, valid_df = model_selection.train_test_split(
    df, test_size=0.2, random_state=42
)

# Data set class for getting data from path

In [ ]:

    
class TaskDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=DATA_PATH, mode="train", transforms=None):
        super().__init__()
        self.df_data = df
        self.data_path = data_path
        self.transforms = transforms
        self.mode = mode
        self.data_dir = TRAIN_PATH if mode == "train" else TEST_PATH

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        img_name = self.df_data.StudyInstanceUID.values[index]
        label = self.df_data[self.df_data.StudyInstanceUID == img_name].values.tolist()[0][1:-1]
        img_path = os.path.join(self.data_dir, img_name + ".jpg" )
        img =   Image.open(img_path).convert("RGB")
        label = torch.tensor(label,dtype= torch.float32) 
        if self.transforms is not None:
            image = self.transforms(img)

        return image, label

# **  transforming data doing augmention **

In [ ]:
# Imagenet means and stds
mean = [0.485, 0.456, 0.406]
std  = [0.229, 0.224, 0.225]
        
transforms_train = transforms.Compose([
                            transforms.Resize((IMG_SIZE, IMG_SIZE)),
                         transforms.RandomRotation(45),
                            transforms.RandomHorizontalFlip(),
                            transforms.RandomVerticalFlip(),
                            transforms.RandomResizedCrop(IMG_SIZE),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=mean, std=std),
                           # transforms.Grayscale(num_output_channels=1)
                        ]
                    )

transforms_valid = transforms.Compose(
                        [
                            transforms.Resize((IMG_SIZE, IMG_SIZE)),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=mean, std=std),
                            #transforms.Grayscale(num_output_channels=1)
                        ]
                    )        



#  make datasets and data loaders

In [ ]:
train_dataset = TaskDataset(train_df, transforms=transforms_train)
valid_dataset = TaskDataset(valid_df, transforms=transforms_valid)

In [ ]:
del df
del train_df
del valid_df

In [ ]:
train_loader =  DataLoader( dataset=train_dataset,
        batch_size=BATCH_SIZE, 
      
        drop_last=True,
        num_workers=8,shuffle=True)
valid_loader =    DataLoader(  dataset=valid_dataset,
        batch_size=val_BATCH_SIZE ,
        
        drop_last=True,
        num_workers=8,shuffle=False)


In [ ]:
inputs, classes = next(iter(train_loader))  
print(classes)

# in these code we defin pytorchlightning model




In [ ]:
class Mamonmodel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        i = 0
        self.lr=1e-3
        self.wd = 1e-6
        self.schd = 'CosineAnnealingWarmRestarts'
        #self.best_model_wts = copy.deepcopy(model.state_dict())
        self.best_loss = np.Inf

        basemodel = torch.hub.load('facebookresearch/deit:main', 'deit_base_patch16_384', pretrained=True)
        for child in basemodel.blocks.children():
            if i < 3:
                for param in child.parameters():
                    param.requires_grad = False
            else:
                for param in child.parameters():
                    param.requires_grad = True
            i +=1
        basemodel.head  =  nn.Sequential( 
                                nn.Dropout(0.25), 
             nn.Linear(768, 11)

                    )
        self.model = basemodel
        
    def forward(self,x):
        x = self.model(x)
        return x
    def training_step(self,batch,batch_idx):
        x , y = batch
        y_hat = self.model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat , y)
        #scor = label_ranking_average_precision_score(y.cpu(), torch.sigmoid(y_hat).cpu())
        self.log('train_loss', loss)
        #self.log('train_score', scor)
        return loss  
    def validation_step(self,batch,batch_idx):
        x , y = batch
        y_hat = self.model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat , y)
        #scor = label_ranking_average_precision_score(y.cpu(), torch.sigmoid(y_hat).cpu())
        #self.log('val_score', scor)
        self.log('val_loss', loss) 
        if self.best_loss > loss:
            self.best_loss = loss
            torch.save(self.model.state_dict(), 'Deit-bestwigh-ep-'+str(self.current_epoch)+'.pth')
        return loss    
    def configure_optimizers(self):
        optimizer = optim.AdamW(
            self.model.parameters(), lr=self.lr, weight_decay=self.wd
        )
        if self.schd == 'ReduceLROnPlateau':
            scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.9, patience=3,
                                          verbose=True)
        elif self.schd == 'CosineAnnealingLR':
            scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0)
        elif self.schd == 'CosineAnnealingWarmRestarts':
            scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=0.001, last_epoch=-1) 

        return {'optimizer': optimizer, 'lr_scheduler': scheduler  ,  'monitor':monit }
        

In [ ]:
# init model
memomodel = Mamonmodel()

In [ ]:
'''
{val_loss:.2f}
'''
early_stop_callback = EarlyStopping(
   monitor='val_loss',
    min_delta=0.00,
   patience=3,
    verbose=False,
   mode='min'
 )
if training == 'gpu':
    checkpoint_callback = ModelCheckpoint(filename='{epoch}-gpumodel.pth', 
    verbose=True,
    monitor='val_loss',
    mode='min')
    trainer = pl.Trainer(min_epochs=1 , max_epochs=epoch, check_val_every_n_epoch=valchecking,
            
                        gpus=-1,callbacks=[checkpoint_callback , early_stop_callback]
    ) 
    
elif training == 'tpu':
    print('now we n tpu')
    checkpoint_callback = ModelCheckpoint(filename='{epoch}-tpumodel.pth',  
    verbose=True,
    monitor=monit,
    mode='min')
    trainer = pl.Trainer(tpu_cores=8,min_epochs=1,num_sanity_val_steps=0 , check_val_every_n_epoch=valchecking , max_epochs=epoch,  callbacks=[checkpoint_callback])
else:
    checkpoint_callback = ModelCheckpoint(filename='{epoch}-cpumodel.pth',  
    verbose=True,
    monitor='val_loss',
    mode='min')
    trainer = pl.Trainer(min_epochs=1 , max_epochs=epoch,check_val_every_n_epoch=valchecking,  callbacks=[checkpoint_callback , early_stop_callback])

    


trainer.fit(memomodel, train_loader , valid_loader)